# 오늘은 LeNet 구조를 만들어봅시다


LeNet 구조는 CNN이며, 초기에 만들어진 모델입니다. 

2가지 모델(Sigmoid, ReLU)를 만들어 두 모델의 성능을 비교해봅시다.


## 1.우선 필요 라이브러리를 import 합니다.

In [66]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets
import torchvision.transforms as transforms

import torch.optim as optim

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인

In [67]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.6.0  Device: cpu


## 3. MNIST 데이터 다운로드 

 1. Training data와 Test data 분리하기
 
 2. Training data를 Training data 와 Validation data로 분리하기

In [68]:
BATCH_SIZE = 64

transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.5),(0.5)),
    ])

train_data = datasets.MNIST('data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('data', train=False, download=True, transform=transform)

train, val = torch.utils.data.random_split(train_data,[int(len(train_data)*0.95),len(train_data)-int(len(train_data)*0.95)])


train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=BATCH_SIZE, shuffle=False)
val_loader = torch.utils.data.DataLoader(dataset=val, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)


## 4. torch.nn을 이용하여 모델-1 만들기

   1) 아래의 그림 중 LeNet 구조를 구현 할 것
   
   2) Sigmoid 활성화 함수를 이용할 것
   
   
![](Comparison_image_neural_networks.svg.png)

In [69]:
import torch.nn as nn

class Model_1(nn.Module):
    def __init__(self):
        super(Model_1, self).__init__()      
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, padding=0)
        self.fc3=nn.Linear(5*5*16,120)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)
    def forward(self,x):
        sigmoid = torch.nn.Sigmoid()
        x = sigmoid(self.conv1(x))
        x = torch.nn.functional.avg_pool2d(x, kernel_size=2,stride=2)
        x = sigmoid(self.conv2(x))
        x = torch.nn.functional.avg_pool2d(x, kernel_size=2, stride=2)
        x = torch.flatten(x,1)
        x = sigmoid(self.fc3(x))
        x = sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

## 5. torch.nn을 이용하여 모델-2 만들기

   LeNet 모델에서 ReLU 활성화 함수를 사용하시요

In [78]:
class Model_2(nn.Module):
    def __init__(self):
        super(Model_2, self).__init__()              
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, padding=0)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)
        self.fc3=nn.Linear(5*5*16,120)
    def forward(self,x):
        relu = nn.ReLU()
        x = relu(self.conv1(x))
        x = torch.nn.functional.avg_pool2d(x, kernel_size=2,stride=2)
        x = relu(self.conv2(x))
        x = torch.nn.functional.avg_pool2d(x, kernel_size=2, stride=2)
        x = torch.flatten(x, 1)
        x= relu(self.fc3(x))
        x = relu(self.fc1(x))
        x = self.fc2(x)
        return x

## 7. 학습 준비하기

1) 1 epoch를 학습할 수 있는 함수 만들기

2) Test와 Validation data의 정확도 계산할 수 있는 함수 만들기

In [71]:
def training_epoch(train_loader, network, loss_func, optimizer, epoch):
    train_losses = []
    train_correct = 0
    log_interval = 300
    
    for batch_idx, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)
        # 미분값의 초기화
        optimizer.zero_grad()
        # Forward propagration 계산하기.
        outputs = network(image)
        
        
        # Cross_entropy 함수를 적용하여 loss를 구하고 저장하기
        loss = loss_func(outputs,label)
        train_losses.append(loss.item())

        # training accuracy 정확도 구하기 위해 맞는 샘플 개수 세기
        pred = outputs.data.max(1, keepdim=True)[1]
        train_correct +=pred.eq(label.view_as(pred)).sum()

        # Gradinet 구하기
        loss.backward()

        # weight값 update 하
        optimizer.step()

        # 학습 상황 출력
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.2f}%)]\tLoss: {:.6f}'
                  .format(epoch, batch_idx * len(label), len(train_loader.dataset),100. * batch_idx / len(train_loader),
                          loss.item()))
            
    return train_losses, train_correct

In [72]:
def test_epoch(test_loader, network, loss_func, val = False):
    correct = 0
    
    test_losses = []
    
    with torch.no_grad():
        for batch_idx, (image, label) in enumerate(test_loader):
            image, label = image.to(device), label.to(device)

            # Forward propagration 계산하기.
            outputs = network(image)
            # Cross_entropy 함수를 적용하여 loss를 구하기
            loss = loss_func(outputs,label)
            test_losses.append(loss.item())

            # Batch 별로 정확도 구하기
            pred = outputs.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.view_as(pred)).sum()

        # 전체 정확도 구하기
        test_accuracy = 100. * correct / len(test_loader.dataset)

        #중간결과 출력
        if val is True:
                print('Validation set: Accuracy: {}/{} ({:.2f}%)\n'
              .format(correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        
        else:
            print('Test set: Accuracy: {}/{} ({:.2f}%)\n'
                  .format(correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        
    return test_losses, test_accuracy


## 8. 위 정의된 함수로 학습 함수 만들기

Adam Optimizer를 사용하여 학습시키기

In [75]:
def training(network, learning_rate = 0.001):
    
    epoches = 15
    
    cls_loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)
    
    train_losses_per_epoch = []
    test_losses_per_epoch = []
    
    train_accuracies = []
    test_accuracies = []
    
    
    for epoch in range(epoches):
                
        # 모델를 학습 중이라고 선언하기
        network.train()
        
        train_losses, train_correct = training_epoch(train_loader,network,cls_loss,optimizer, epoch)
        
        # epoch 별로 loss 평균값, 정확도 구하기
        average_loss = np.mean(train_losses)
        train_losses_per_epoch.append(average_loss)
        
        train_accuracy = torch.true_divide(train_correct,len(train_loader.dataset)) * 100
        train_accuracies.append(train_accuracy)
        
        # epoch 별로 정확도 출력
        print('\nTraining set: Accuracy: {}/{} ({:.2f}%)'
              .format(train_correct, len(train_loader.dataset),100. * train_correct / len(train_loader.dataset)))

        
        ### 학습 중에 test 결과 보기
        
        # 모델 test 중인 것을 선언하기
        network.eval()
        
        correct = 0
        with torch.no_grad():
            test_losses, test_accuracy = test_epoch(val_loader, network, cls_loss, True)

        test_losses_per_epoch.append(np.mean(test_losses))
        test_accuracies.append(test_accuracy)
        
    with torch.no_grad():
        test_losses, test_accuracy = test_epoch(test_loader, network, cls_loss, False)
        
    return train_losses_per_epoch, test_losses_per_epoch, train_accuracies, test_accuracies


In [76]:
network = Model_1().to(device)
rlt_const = training(network)

Train Epoch: 0 [0/57000 (0.00%)]	Loss: 2.366459
Train Epoch: 0 [19200/57000 (33.67%)]	Loss: 1.184591
Train Epoch: 0 [38400/57000 (67.34%)]	Loss: 0.576667

Training set: Accuracy: 37767/57000 (66.26%)
Validation set: Accuracy: 2736/3000 (91.20%)

Train Epoch: 1 [0/57000 (0.00%)]	Loss: 0.344640
Train Epoch: 1 [19200/57000 (33.67%)]	Loss: 0.298701
Train Epoch: 1 [38400/57000 (67.34%)]	Loss: 0.355750

Training set: Accuracy: 52998/57000 (92.98%)
Validation set: Accuracy: 2821/3000 (94.03%)

Train Epoch: 2 [0/57000 (0.00%)]	Loss: 0.208146
Train Epoch: 2 [19200/57000 (33.67%)]	Loss: 0.189328
Train Epoch: 2 [38400/57000 (67.34%)]	Loss: 0.241919

Training set: Accuracy: 54244/57000 (95.16%)
Validation set: Accuracy: 2863/3000 (95.43%)

Train Epoch: 3 [0/57000 (0.00%)]	Loss: 0.138699
Train Epoch: 3 [19200/57000 (33.67%)]	Loss: 0.145489
Train Epoch: 3 [38400/57000 (67.34%)]	Loss: 0.164559

Training set: Accuracy: 54905/57000 (96.32%)
Validation set: Accuracy: 2884/3000 (96.13%)

Train Epoch: 4 [

In [79]:
network = Model_2().to(device)
rlt_const = training(network)

Train Epoch: 0 [0/57000 (0.00%)]	Loss: 2.304616
Train Epoch: 0 [19200/57000 (33.67%)]	Loss: 0.226073
Train Epoch: 0 [38400/57000 (67.34%)]	Loss: 0.150618

Training set: Accuracy: 51939/57000 (91.12%)
Validation set: Accuracy: 2895/3000 (96.50%)

Train Epoch: 1 [0/57000 (0.00%)]	Loss: 0.130214
Train Epoch: 1 [19200/57000 (33.67%)]	Loss: 0.084177
Train Epoch: 1 [38400/57000 (67.34%)]	Loss: 0.052022

Training set: Accuracy: 55642/57000 (97.62%)
Validation set: Accuracy: 2929/3000 (97.63%)

Train Epoch: 2 [0/57000 (0.00%)]	Loss: 0.100458
Train Epoch: 2 [19200/57000 (33.67%)]	Loss: 0.066236
Train Epoch: 2 [38400/57000 (67.34%)]	Loss: 0.021273

Training set: Accuracy: 56015/57000 (98.27%)
Validation set: Accuracy: 2938/3000 (97.93%)

Train Epoch: 3 [0/57000 (0.00%)]	Loss: 0.092957
Train Epoch: 3 [19200/57000 (33.67%)]	Loss: 0.069989
Train Epoch: 3 [38400/57000 (67.34%)]	Loss: 0.012819

Training set: Accuracy: 56203/57000 (98.60%)
Validation set: Accuracy: 2950/3000 (98.33%)

Train Epoch: 4 [

## 9. 두모델의 성능을 비교하시오

정답) relu가 sigmoid에 비해 vanishing gradient가 적기때문에 더 뛰어난 성능을 보인다. relu가 sigmoid에 비해 빠르게 좋은 성능으로 수렴한다.